# Système de recommendation avec KNN

<div align="center">
<div><img width="80%" height="80%" src="./images/cover.png" title="Système de recommendation avec KNN"/></div>
<div><a href="https://humboldt-wi.github.io/blog/research/information_systems_1819/is_lda_final/">Source de l'image</a> </div>
</div>

# Introduction

Pratiquement tout le monde a vécu une expérience en ligne où un site Web fait des recommandations personnalisées pour chaque client dans l'espoir d'avoir plus de vente et mieux satisfaire la demande. *Netflix* a décerné un prix d'un million de dollars à une équipe de développeurs en 2009, pour un algorithme qui a augmenté la précision du système de recommandation de l'entreprise de 10 %.

# Données

[**Book-Crossings**](http://www2.informatik.uni-freiburg.de/~cziegler/BX/) est un ensemble de données sur les notes de livres compilé par Cai-Nicolas Ziegler. Il contient 1,1 million d'évaluations de 270 000 livres par 90 000 utilisateurs. Les notes sont sur une échelle de 1 à 10.

Les données se composent de trois tableaux repartis dans trois fichier au format CSV :
- `BX-Book-Ratings.csv` qui comporte les données d'évaluations;
- `BX-Books.csv` qui comporte les informations sur les livres;
- `BX-Users.csv` qui comporte les informations sur les utilisateurs.

Pour télécharger et décompresser ces trois (3) qui constituent l'ensemble de données, il faut juste exécuter les deux commandes suivantes qui va nous télécharger un fichier compressé se trouvant à cette [adresse](https://cdn.freecodecamp.org/project-data/books/book-crossings.zip).

```shell
wget -c https://cdn.freecodecamp.org/project-data/books/book-crossings.zip && \
unzip book-crossings.zip -d data
```
Voici le code pour un noteboock :

In [6]:
!wget -c https://cdn.freecodecamp.org/project-data/books/book-crossings.zip && \
unzip book-crossings.zip -d data

--2023-08-04 03:56:53--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  49.8KB/s    in 11m 59s 

2023-08-04 04:08:54 (35.4 KB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: data/BX-Book-Ratings.csv  
  inflating: data/BX-Books.csv       
  inflating: data/BX-Users.csv       


On va utiliser la librairy Pandas pour charger les fichiers.

In [13]:
import os
import pandas as pd


DATA_DIR = './data'
books_filename = os.path.join(DATA_DIR, 'BX-Books.csv')
users_filename = os.path.join(DATA_DIR, 'BX-Users.csv')
ratings_filename = os.path.join(DATA_DIR, 'BX-Book-Ratings.csv')

df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_users = pd.read_csv(
    users_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    usecols=['user_id', 'location', 'age'],
    names=['user_id', 'location', 'age'],
    dtype={'user_id': 'int32', 'location': 'str'}
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)

print(f"Books:\n{df_books.head()}")
print(f"Users:\n{df_users.head()}")
print(f"Evaluations:\n{df_ratings.head()}")

Books:
         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  
Users:
   user_id                            location   age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN
Evaluations:
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0


L'ensemble de données d'évaluation `df_ratings` fournit une liste des évaluations que les utilisateurs ont attribuées aux livres. Il comprend 1 149 780 enregistrements et 3 colonnes : 
- `userID` renommé en `user`;
- `ISBN` renommé en `isbn`;
- et `bookRating` renommé en `ratting`.

In [15]:
print(df_ratings.shape)  # affiche le nombre de lignes et colonnes.
print(list(df_ratings))  # affiche la liste des colonnes.

(1149780, 3)
['user', 'isbn', 'rating']
